# Shift the source Position around the ceentral lens

- creation date : 2025-06-22
- last update : 2025-06-26

- https://arxiv.org/pdf/1803.09746

- https://lenstronomy.readthedocs.io/en/latest/

- https://colab.research.google.com/github/lenstronomy/lenstronomy-tutorials/blob/main/Notebooks/GettingStarted/starting_guide.ipynb#scrollTo=32isZ3l7fPXj

In [ ]:
# some standard python imports #
import numpy as np
import copy
import matplotlib.pyplot as plt
%matplotlib inline

# make sure lenstronomy is installed, otherwise install the latest pip version
try:
    import lenstronomy
except:
    print("!pip install lenstronomy")
    
print("This notebook is running with lenstronomy version", lenstronomy.__version__)

# *lenstronomy* starting guide

Welcome to lenstronomy!

This notebook aims to describe the basic functionalities of the most important modules of *lenstronomy* with some simple examples. More detailed information about the available routines and their use can be accessed through the [online documentation](https://readthedocs.com/lenstronomy) and in many other [Jupyter notebooks](https://github.com/sibirrer/lenstronomy_extensions).

Also, we refer to the [strong lensing lecture notes](https://github.com/sibirrer/strong_lensing_lectures) in Jupyter form for a brief introduction in strong gravitational lensing.


<div class="alert alert-warning">

**Disclamer:** Please reach out to the developers if you have questions! This is true for a beginner in gravitational lensing or have substantial experience with other lensing codes. Your questions and feedback is very valuable for us to improve the software and its documentation.

</div>

## Configuration

### LensModel module

 $\texttt{LensModel}$ module
$\texttt{LensModel}$ and its sub-packages execute all the purely lensing related tasks of *lenstronomy*. This includes ray-shooting, solving the lens equation, arrival time computation and non-linear solvers to optimize lens models for specific image configurations. The module allows consistent integration with single and multi plane lensing and an arbitrary superpositions of lens models. There is a wide range of lens models available. For details we refer to to the online-documentation.

#### Single plane lensing
As an example of initializing a single plane lens model as a super-position of an elliptical power-law potential and a singular isothermal sphere mass distribution and execute some basic routines:

In [ ]:
# import the LensModel class #
from lenstronomy.LensModel.lens_model import LensModel

# specify the choice of lens models #
lens_model_list = ['EPL', 'SHEAR']

# setup lens model class with the list of lens models #
lensModel = LensModel(lens_model_list=lens_model_list)

# define parameter values of lens models #

kwargs_spep = {'theta_E': 1.1, 'e1': 0.01, 'e2': -0.01, 'gamma': 2., 'center_x': 0.0, 'center_y': 0}
kwargs_shear = {'gamma1': -0.00, 'gamma2': .0}
kwargs_lens = [kwargs_spep, kwargs_shear]

In [ ]:
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver
# specifiy the lens model class to deal with #
solver = LensEquationSolver(lensModel)

In [ ]:
beta_ra = np.linspace(-1.2,1.2,5)
beta_dec = np.linspace(1.2,-1.2,5)
beta_radec = np.meshgrid(beta_ra,beta_dec )
NRA = len(beta_ra)
NDEC = len(beta_dec)

In [ ]:
props = dict(boxstyle='round',facecolor='white',alpha=0.1)

In [ ]:
from lenstronomy.Plots import lens_plot
fig,axs = plt.subplots(NRA,NDEC,figsize = (18,18),layout="constrained")
for j_beta_ra in range(NRA):
    for i_beta_dec in range(NDEC):   
        beta_ra = beta_radec[0][j_beta_ra,i_beta_dec]
        beta_dec = beta_radec[1][j_beta_ra,i_beta_dec]
        #print(beta_ra,beta_dec)

        textstr = '\n'.join((f"ra = {beta_ra:.1f} as",f"dec = {beta_dec:.1f} as"))
        
        # solve for image positions provided a lens model and the source position #
        theta_ra, theta_dec = solver.image_position_from_source(beta_ra, beta_dec, kwargs_lens)

        

        # the magnification of the point source images #
        mag = lensModel.magnification(theta_ra, theta_dec, kwargs_lens)

        ax = axs[j_beta_ra,i_beta_dec]
        lens_plot.lens_model_plot(ax, lensModel=lensModel, kwargs_lens=kwargs_lens, sourcePos_x=beta_ra, sourcePos_y=beta_dec, point_source=True, with_caustics=True, fast_caustic=True, coord_inverse=False)
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,verticalalignment='top', bbox=props)

plt.show()